In [57]:
import sys
sys.path.append("../")
from tensorflow import keras
from src.preprocess.word_embedding_features import sentence_level_preprocess
import pandas as pd
import gensim.downloader
import re
import swifter
from sklearn.model_selection import train_test_split
from keras.layers import GRU, AvgPool1D, Dense, Masking
from keras.models import Sequential, load_model

In [ ]:
embedding_table = gensim.downloader.load("glove-wiki-gigaword-50")
model = keras.models.load_model('outputs/sentence_level_preprocess-checkpoints')
df = pd.read_csv("../data/data.csv",nrows=5000)

In [3]:
df["author"] = pd.Categorical(df["author"]).codes
df = df[(df["author"] == 2512) | (df["author"]==2199)][["author","Text"]]
df = df.reset_index()
df["Text"] = df["Text"].apply(lambda text: re.sub('[^A-Za-z0-9\n\s]+', '', text).replace("\n\n","\n"))
X = df["Text"].swifter.apply(lambda text: sentence_level_preprocess(text,embedding_table))
y = df["author"].values
y[y == 2512] = 0
y[y == 2199] = 1
one_hot = keras.utils.to_categorical(y, num_classes=2, dtype="float32")
y = np.expand_dims(one_hot, axis=1)
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2)

In [58]:
new_model = Sequential()
new_model.add(Masking(mask_value=0.0, input_shape=(170, 50),))
new_model.add(
    GRU(
        100,
        recurrent_dropout=0.2,
        input_shape=(170, 50),
        return_sequences=True,
    )
)
new_model.add(AvgPool1D(pool_size=(170,)))
new_model.add(Dense(1, activation="sigmoid"))
new_model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

In [59]:
new_model.summary()
new_model.layers[1].set_weights(model.layers[0].trainable_weights)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_8 (Masking)          (None, 170, 50)           0         
_________________________________________________________________
gru_8 (GRU)                  (None, 170, 100)          45600     
_________________________________________________________________
average_pooling1d_8 (Average (None, 1, 100)            0         
_________________________________________________________________
dense_8 (Dense)              (None, 1, 1)              101       
Total params: 45,701
Trainable params: 45,701
Non-trainable params: 0
_________________________________________________________________


TypeError: __array__() takes 1 positional argument but 2 were given

In [27]:
history = model.fit(np.stack(X_train.values), y_train, epochs=2, shuffle=True,batch_size=32)

Epoch 1/2


ValueError: in user code:

    C:\Users\yosef\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\yosef\AppData\Roaming\Python\Python38\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\yosef\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\yosef\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\yosef\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\yosef\AppData\Roaming\Python\Python38\sit

In [ ]:
model.predict().argmax(axis=-1).flatten()